In [28]:
%pylab inline
import sep, os
from astropy.io import fits

Populating the interactive namespace from numpy and matplotlib


In [40]:
path_imagenes = './imagenes/deteccion_fuentes/'

In [3]:
data = loadtxt('./imagenes/graficas/media.dat')

print shape(data)

(4612, 2048)


In [4]:
#Background

bkg = sep.Background(data)

In [24]:
#Source detection

thresh = 10.0 * bkg.globalrms
objects = sep.extract(data, thresh) # objects is a numpy structured array

num_objects = len(objects)
print num_objects # number of objects

654


In [15]:
objects_x = []
objects_y = []

for i in range(num_objects):
    x = objects['x'][i]
    y = objects['y'][i]
    
    objects_x.append(x)
    objects_y.append(y)
    
objects_x = array(objects_x)
objects_y = array(objects_y)

In [16]:
x_max = 2048
y_max = 4612

fig = figure(figsize=(14,24))
imshow(data, cmap='gray',vmin=180,vmax=200)
colorbar()

for i in range(len(objects)):
    x = objects_x[i]
    y = objects_y[i]
    
    porc_x = x/float(x_max)
    porc_y = 1-y/float(y_max)
    
    axvline(x,porc_y-0.01,porc_y+0.01)
    axhline(y,porc_x-0.02,porc_x+0.02) 
    
savefig(path_imagenes+'fuentes.png', bbox_inches='tight')
close()

#Curvas de luz

In [25]:
path_s = '../data_marcosmo/2006-02-06/'
path_o = '../data_marcosmo/2006-06-30/'
visitas = []

for v_s in os.listdir(path_s):
    visitas.append(path_s+v_s)
    
for v_o in os.listdir(path_o):
    visitas.append(path_o+v_o)

visitas = sort(array(visitas))
num_visitas = len(visitas)

print num_visitas
print visitas

12
['../data_marcosmo/2006-02-06/calexp-837002-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837003-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837004-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837005-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837006-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837007-14.fits'
 '../data_marcosmo/2006-02-06/calexp-837008-14.fits'
 '../data_marcosmo/2006-06-30/calexp-853539-14.fits'
 '../data_marcosmo/2006-06-30/calexp-853540-14.fits'
 '../data_marcosmo/2006-06-30/calexp-853541-14.fits'
 '../data_marcosmo/2006-06-30/calexp-853542-14.fits'
 '../data_marcosmo/2006-06-30/calexp-853543-14.fits']


In [33]:
objects_x_int = array([int(ox) for ox in objects_x])
objects_y_int = array([int(oy) for oy in objects_y])

In [26]:
curvas = zeros((num_objects,num_visitas))

In [34]:
for v in range(num_visitas):
            
    visita = fits.open(visitas[v])
    ccd14 = visita[1].data
    #ccd14 = ccd14*65000/amax(ccd14) #Normalizando, no estoy segura si debería ir ahi
       
    for i in range(num_objects):
        
        columna = objects_x_int[i]
        fila = objects_y_int[i]
        
        pixel = ccd14[fila][columna]
            
        curvas[i][v] = pixel

    visita.close()

In [47]:
vis = arange(num_visitas)

for i in range(num_objects):
    intensidad = curvas[i][:]
    
    fig = figure(figsize=(20,10))
    scatter(vis,intensidad)
    plot(vis,intensidad,c='c')
    xlabel('Visita',fontsize=20)
    ylabel('Intensidad',fontsize=20)
    xlim(0,num_visitas-1)
    ylim(-amax(intensidad)*0.01,amax(intensidad)*1.01)
    savefig(path_imagenes+'curvas_de_luz/'+str(i)+'.png')
    close()